# python中文新闻分类

In [28]:
import re  
import pickle
import random
import jieba
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer as TFIV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

### 没有预处理，直接分词

In [ ]:
### 导入数据、打乱数据顺序
data = [[i.split("\t")[0],i.split("\t")[1]] for i in open("F:/项目/新闻文本分类/data/cnews.train.txt","r",encoding='utf-8')]
random.shuffle(data)
#jieba分词
text = [" ".join(jieba.lcut(i[1])) for i in data]
#使用标签编码的方式将label转化为数字
lbl_enc = preprocessing.LabelEncoder()
lable_y = [i[0] for i in data]
y = lbl_enc.fit_transform(lable_y)
#使用TF-IDF对词向量进行表示。存储为稀疏矩阵格式
tfv = TFIV(min_df=3,  max_features=None, strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}', ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1, stop_words = 'english')
tfv.fit(text)
X_all = tfv.transform(text)
len_train = 6000
X_train = X_all[:len_train]
y_train = lable_y[:len_train]
X_test = X_all[len_train:]
y_test = lable_y[len_train:]

### 数据预处理

In [29]:
import jieba
import pandas as pd
df_cnews =pd.read_csv("./data/cnews.train.txt",sep="\t",names=['category','cnews'], encoding='utf-8')
df_cnews = df_cnews.dropna()
# random.shuffle(df_cnews)
# 停用词
stopwords=pd.read_csv("./data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values
# 构建数据集
def preprocess_text(df_cnews, sentences):
    for i,text in enumerate(df_cnews['cnews']):
        try:
            segs=jieba.lcut(text)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append((" ".join(segs), df_cnews['category'][i]))
        except Exception as e:
            print(line)
            continue 

#生成训练数据
sentences = []
preprocess_text(df_cnews,sentences)

from sklearn.model_selection import train_test_split
text, lable_y = zip(*sentences)

## 多分类

### 机器学习

In [ ]:
#使用TF-IDF对词向量进行表示
y = lbl_enc.fit_transform(lable_y)
#存储为稀疏矩阵格式
tfv = TFIV(min_df=3,  max_features=None, strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}', ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1, stop_words = 'english')
tfv.fit(text)
x = tfv.transform(text)
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=2019)

LR

In [39]:
# model_LR = LogisticRegression(C=.01) # C是正则化系数。
# model_LR.fit(X, y_train)
# print("训练集20折交叉验证得分: ", np.mean(cross_val_score(model_LR, X, y_train, cv=20)))
# # model_LR.transform(X_test,y_test)
# print("测试集20折交叉验证得分: ", np.mean(cross_val_score(model_LR, X_test, y_test, cv=20)))

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


训练集20折交叉验证得分:  0.8830281811182141


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


测试集20折交叉验证得分:  0.8550458987148817


In [97]:
model_LR = LogisticRegression(C=.01) # C是正则化系数。
model_LR.fit(X, y_train)
print("训练集20折交叉验证得分: ", np.mean(cross_val_score(model_LR, X, y_train, cv=20)))
# model_LR.transform(X_test,y_test)
print("测试集20折交叉验证得分: ", np.mean(cross_val_score(model_LR, X_test, y_test, cv=20)))

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


训练集20折交叉验证得分:  0.9620329965014577


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


测试集20折交叉验证得分:  0.8106312926667842


svm

In [92]:
model_SVM = LinearSVC(C=.01) # C是正则化系数。
model_SVM.fit(X_train, y_train)
print("训练集20折交叉验证得分: ", np.mean(cross_val_score(model_SVM, X_train, y_train, cv=20)))
# model_LR.transform(X_test,y_test)
print("测试集20折交叉验证得分: ", np.mean(cross_val_score(model_SVM, X_test, y_test, cv=20)))

训练集20折交叉验证得分:  0.9576807168048722
测试集20折交叉验证得分:  0.9655212518111689


In [98]:
model_SVM = LinearSVC(C=.01) # C是正则化系数。
model_SVM.fit(X_train, y_train)
print("训练集20折交叉验证得分: ", np.mean(cross_val_score(model_SVM, X_train, y_train, cv=20)))
# model_LR.transform(X_test,y_test)
print("测试集20折交叉验证得分: ", np.mean(cross_val_score(model_SVM, X_test, y_test, cv=20)))

训练集20折交叉验证得分:  0.9713594872280804
测试集20折交叉验证得分:  0.9690510982207394


### 神经网络

CNN

In [21]:
"""
基于卷积神经网络的中文文本分类
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import pandas
from sklearn import metrics
import tensorflow as tf

learn = tf.contrib.learn

FLAGS = None

#文档最长长度
MAX_DOCUMENT_LENGTH = 100
#最小词频数
MIN_WORD_FREQUENCE = 2
#词嵌入的维度
EMBEDDING_SIZE = 20
#filter个数
N_FILTERS = 10
#感知野大小
WINDOW_SIZE = 20
#filter的形状
FILTER_SHAPE1 = [WINDOW_SIZE, EMBEDDING_SIZE]
FILTER_SHAPE2 = [WINDOW_SIZE, N_FILTERS]
#池化
POOLING_WINDOW = 4
POOLING_STRIDE = 2
n_words = 0

In [22]:
#使用词袋模型对词向量进行表示
y = lbl_enc.fit_transform(lable_y)
X_train, X_test, y_train, y_test = train_test_split(text, y, random_state=2019)

NameError: name 'lable_y' is not defined

In [ ]:
global n_words
# 处理词汇
vocab_processor = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH, min_frequency=MIN_WORD_FREQUENCE)
x_train = np.array(list(vocab_processor.fit_transform(X_train)))
x_test = np.array(list(vocab_processor.transform(X_test)))
n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

In [11]:
def cnn_model(features, target):
    """
    2层的卷积神经网络，用于短文本分类
    """
    # 先把词转成词嵌入
    # 我们得到一个形状为[n_words, EMBEDDING_SIZE]的词表映射矩阵
    # 接着我们可以把一批文本映射成[batch_size, sequence_length, EMBEDDING_SIZE]的矩阵形式
    target = tf.one_hot(target, 15, 1, 0)
    word_vectors = tf.contrib.layers.embed_sequence(
            features, vocab_size=n_words, embed_dim=EMBEDDING_SIZE, scope='words')
    word_vectors = tf.expand_dims(word_vectors, 3)
    with tf.variable_scope('CNN_Layer1'):
        # 添加卷积层做滤波
        conv1 = tf.contrib.layers.convolution2d(
                word_vectors, N_FILTERS, FILTER_SHAPE1, padding='VALID')
        # 添加RELU非线性
        conv1 = tf.nn.relu(conv1)
        # 最大池化
        pool1 = tf.nn.max_pool(
                conv1,
                ksize=[1, POOLING_WINDOW, 1, 1],
                strides=[1, POOLING_STRIDE, 1, 1],
                padding='SAME')
        # 对矩阵进行转置，以满足形状
        pool1 = tf.transpose(pool1, [0, 1, 3, 2])
    with tf.variable_scope('CNN_Layer2'):
        # 第2个卷积层
        conv2 = tf.contrib.layers.convolution2d(
                pool1, N_FILTERS, FILTER_SHAPE2, padding='VALID')
        # 抽取特征
        pool2 = tf.squeeze(tf.reduce_max(conv2, 1), squeeze_dims=[1])

    # 全连接层
    logits = tf.contrib.layers.fully_connected(pool2, 15, activation_fn=None)
    loss = tf.losses.softmax_cross_entropy(target, logits)

    train_op = tf.contrib.layers.optimize_loss(
            loss,
            tf.contrib.framework.get_global_step(),
            optimizer='Adam',
            learning_rate=0.01)

    return ({
            'class': tf.argmax(logits, 1),
            'prob': tf.nn.softmax(logits)
    }, loss, train_op)

In [16]:
# 构建模型
classifier = learn.SKCompat(learn.Estimator(model_fn=cnn_model))

# 训练和预测
classifier.fit(x_train, y_train, steps=3000)
y_predicted = classifier.predict(x_test)['class']
score = metrics.accuracy_score(y_test, y_predicted)
print('Accuracy: {0:f}'.format(score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002504D9B7630>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmp3by0sd2y'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving 

RNN---GRU

In [30]:
"""
使用RNN完成文本分类
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import pandas 
from sklearn import metrics
import tensorflow as tf
from tensorflow.contrib.layers.python.layers import encoders
learn = tf.contrib.learn

FLAGS = None

In [32]:
MAX_DOCUMENT_LENGTH = 15
MIN_WORD_FREQUENCE = 2
EMBEDDING_SIZE = 50
global n_words

# 使用词袋模型对词向量进行表示
y = lbl_enc.fit_transform(lable_y)
X_train, X_test, y_train, y_test = train_test_split(text, y, random_state=2019)

global n_words
# 处理词汇
vocab_processor = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH, min_frequency=MIN_WORD_FREQUENCE)
x_train = np.array(list(vocab_processor.fit_transform(X_train)))
x_test = np.array(list(vocab_processor.transform(X_test)))
n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

Total words: 51967


In [37]:
# GRU分类器
def rnn_model(features,target):
    word_vectors = tf.contrib.layers.embed_sequence(
            features,vocab_size=n_words,embed_dim=EMBEDDING_SIZE,scope='words')
    word_list = tf.unstack(word_vectors,axis=1)
    cell = tf.contrib.rnn.GRUCell(EMBEDDING_SIZE)
    _,encoding = tf.contrib.rnn.static_rnn(cell,word_list,dtype=tf.float32)
    target = tf.one_hot(target,15,1,0)
    logits = tf.contrib.layers.fully_connected(encoding,15,activation_fn=None)
    loss = tf.contrib.losses.softmax_cross_entropy(logits,target)
    
    #优化
    train_op = tf.contrib.layers.optimize_loss(
            loss,
            tf.contrib.framework.get_global_step(),
            optimizer='Adam',
            learning_rate=0.01)
    
    return({
        'class':tf.argmax(logits,1),
        'prob':tf.nn.softmax(logits)
    },loss,train_op)  

In [38]:
model_fn = rnn_model
classifier = learn.SKCompat(learn.Estimator(model_fn=model_fn))
classifier.fit(x_train,y_train,steps=1000)
y_predicted = classifier.predict(x_test)['class']
score = metrics.accuracy_score(y_test,y_predicted)
print('Accuracy: {0:f}'.format(score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025312E7F898>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmpcm59awns'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving 

词袋模型

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import pandas
from sklearn import metrics
import tensorflow as tf
from tensorflow.contrib.layers.python.layers import encoders

learn = tf.contrib.learn

FLAGS = None

In [ ]:
MAX_DOCUMENT_LENGTH = 15
MIN_WORD_FREQUENCE = 2
EMBEDDING_SIZE = 50
global n_words

# 使用词袋模型对词向量进行表示
y = lbl_enc.fit_transform(lable_y)
X_train, X_test, y_train, y_test = train_test_split(text, y, random_state=2019)

global n_words
# 处理词汇
vocab_processor = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH, min_frequency=MIN_WORD_FREQUENCE)
x_train = np.array(list(vocab_processor.fit_transform(X_train)))
x_test = np.array(list(vocab_processor.transform(X_test)))
n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

In [40]:
def bag_of_words_model(features, target):
    """先转成词袋模型"""
    target = tf.one_hot(target, 15, 1, 0)
    features = encoders.bow_encoder(
            features, vocab_size=n_words, embed_dim=EMBEDDING_SIZE)
    logits = tf.contrib.layers.fully_connected(features, 15, activation_fn=None)
    loss = tf.contrib.losses.softmax_cross_entropy(logits, target)
    train_op = tf.contrib.layers.optimize_loss(
            loss,
            tf.contrib.framework.get_global_step(),
            optimizer='Adam',
            learning_rate=0.01)
    return ({
            'class': tf.argmax(logits, 1),
            'prob': tf.nn.softmax(logits)
    }, loss, train_op)


model_fn = bag_of_words_model
classifier = learn.SKCompat(learn.Estimator(model_fn=model_fn))

# Train and predict
classifier.fit(x_train, y_train, steps=1000)
y_predicted = classifier.predict(x_test)['class']
score = metrics.accuracy_score(y_test, y_predicted)
print('Accuracy: {0:f}'.format(score))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000250510DD978>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmpl5nqidmq'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving 

## 二分类

多项式朴素贝叶斯

In [40]:
from sklearn.naive_bayes import MultinomialNB as MNB
# 因为sklearn的NB只支持二分类，所以就简单粗暴地把10个类变成一个类。
y_train_nb = [1 if i > 5 else 0 for i in y_train]
model_NB = MNB()
model_NB.fit(X, y_train_nb ) #特征数据直接灌进来
MNB(alpha=1.0, class_prior=None, fit_prior=True) # ”alpha“是平滑参数，不需要掌握哈。

from sklearn.model_selection import cross_val_score
import numpy as np

print("多项式贝叶斯分类器20折交叉验证得分: ", np.mean(cross_val_score(model_NB, X, y_train_nb, cv=20, scoring='roc_auc')))

多项式贝叶斯分类器20折交叉验证得分:  0.9930848014789697


逻辑回归

In [41]:
from sklearn.linear_model import LogisticRegression as LR
model_LR = LogisticRegression(C=.01) # C是正则化系数。
model_LR.fit(X, y_train_nb)
print("20折交叉验证得分: ", np.mean(cross_val_score(model_LR, X, y_train_nb, cv=20, scoring='roc_auc')))

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


20折交叉验证得分:  0.9850482651479215


SVM

In [42]:

from sklearn.svm import LinearSVC
model_SVM = LinearSVC(C=.01) # C是正则化系数。
model_SVM.fit(X, y_train_nb)
print("20折交叉验证得分: ", np.mean(cross_val_score(model_SVM, X, y_train_nb, cv=20, scoring='roc_auc')))

20折交叉验证得分:  0.9914719455415332
